In [3]:
!pip install plotly==4.8.0 

  Attempting uninstall: plotly
    Found existing installation: plotly 5.2.2
    Uninstalling plotly-5.2.2:
      Successfully uninstalled plotly-5.2.2


In [22]:
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string
import regex as re
import zipfile
import gc
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [6]:
path_to_archive = r"D:\Papers\Paper 3 - Recommender Systems\archive\\"

projects = pd.read_csv(path_to_archive + 'Projects.csv')
schools = pd.read_csv(path_to_archive + 'Schools.csv')
#teachers = pd.read_csv(path_to_archive + 'Teachers.csv')
donations = pd.read_csv(path_to_archive + 'Donations.csv')
#resources = pd.read_csv('Resources.csv')
donors = pd.read_csv(path_to_archive + 'Donors.csv')

C:\Users\Lenovo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
projects.head(2)

,Project ID,School ID,Teacher ID,Teacher Project Posted Sequence,Project Type,Project Title,Project Essay,Project Short Description,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
0,7685f0265a19d7b52a470ee4bac883ba,e180c7424cb9c68cb49f141b092a988f,4ee5200e89d9e2998ec8baad8a3c5968,25,Teacher-Led,Stand Up to Bullying: Together We Can!,Did you know that 1-7 students in grades K-12 ...,Did you know that 1-7 students in grades K-12 ...,"My students need 25 copies of ""Bullying in Sch...",Applied Learning,"Character Education, Early Development",Grades PreK-2,Technology,361.80,2013-01-01,2013-05-30,Fully Funded,2013-01-11
1,f9f4af7099061fb4bf44642a03e5c331,08b20f1e2125103ed7aa17e8d76c71d4,cca2d1d277fb4adb50147b49cdc3b156,3,Teacher-Led,Learning in Color!,"Help us have a fun, interactive listening cent...","Help us have a fun, interactive listening cent...","My students need a listening center, read alon...","Applied Learning, Literacy & Language","Early Development, Literacy",Grades PreK-2,Technology,512.85,2013-01-01,2013-05-31,Expired,NaN


In [8]:
projects.isna().sum()

Project ID                               0
School ID                                0
Teacher ID                               0
Teacher Project Posted Sequence          0
Project Type                             0
Project Title                            6
Project Essay                            1
Project Short Description                3
Project Need Statement                   3
Project Subject Category Tree           29
Project Subject Subcategory Tree        29
Project Grade Level Category             0
Project Resource Category               36
Project Cost                             0
Project Posted Date                      0
Project Expiration Date                 14
Project Current Status                   0
Project Fully Funded Date           283253
dtype: int64

In [12]:
projects = projects.dropna(subset=['Project Title',
                        'Project Essay',
                        'Project ID',
                        'Project Need Statement',
                        'Project Posted Date',
                        'Project Expiration Date',
                        'Project Resource Category',
                        'Project Fully Funded Date'
                        ]).reset_index(drop=True)

In [16]:
assert projects.isna().sum().all() == 0

In [18]:
projects = projects[projects['Project Type'] == 'Teacher-Led'].reset_index(drop=True)

In [19]:
projects[['Project Cost', 'Project Resource Category']].groupby(by='Project Resource Category').sum().sort_values('Project Cost', ascending=False) .head()

,Project Cost
Project Resource Category,
Technology,1.679534e+08
Supplies,1.601576e+08
Books,7.434509e+07
Other,2.961288e+07
Computers & Tablets,2.005473e+07


In [23]:
# Cool, now let's see which project categories are more popular! Which ones are they also costly?
df_exp_3 =  projects[['Project Cost', 'Project Resource Category', 'Project ID']].groupby(by='Project Resource Category').agg({'Project Cost': 'sum', 'Project ID': 'count'}).sort_values(by=('Project ID'))


fig = px.scatter(x = df_exp_3.index,
             y = df_exp_3[('Project ID')],
             size = df_exp_3[('Project Cost')],
             labels=dict(x="Resource Category Type", y="Number of Projects"),
             title="What Projects Have more Popularity and Cost? (radial:cost)")
fig.show()
# love how books are both popular and cheap in comparison to others :)

In [24]:
important_categories = projects[['Project Cost', 'Project Resource Category', 'Project ID']].groupby(by='Project Resource Category').agg({'Project Cost': 'sum', 'Project ID': 'count'}).sort_values('Project ID', ascending=False) .head().index.values.tolist()
important_categories

['Supplies', 'Technology', 'Books', 'Other', 'Computers & Tablets']

In [25]:
projects = projects[projects['Project Resource Category'].isin(important_categories)].reset_index(drop=True)

In [ ]:
donations_projects.drop(
    columns = [
            'School County', 'School District', 'School City',
            'School Percentage Free Lunch', 'School Name', 'Teacher ID', 
            'Donation Included Optional Donation',
            'Project Essay', 'Project Short Description', 'School Percentage Free Lunch',
            'Donor Cart Sequence', 'Teacher Project Posted Sequence'

            
        ], inplace=True
        )

In [27]:
# which grades are more active? Which submit more costly projects?

df_exp_1 = projects[['Project Cost', 'Project ID', 'Project Grade Level Category']].groupby('Project Grade Level Category').agg(['count', 'mean']).reset_index().sort_values('Project Grade Level Category').set_index('Project Grade Level Category')


fig = go.Figure(go.Barpolar(
    theta=df_exp_1.index,\
    r=df_exp_1[('Project Cost', 'count')],
    width=df_exp_1[('Project Cost', 'mean')]/1000,
    marker_color=['gold', 'mediumturquoise', 'darkorange', 'lightgreen', 'pink'],
    marker_line_color="black",
    marker_line_width=2,
    opacity=0.8
))
fig.update_layout(title="How Much each Grade spent on Average? (radial:count, width:Average cost)")

fig.show()

In [28]:
projects = projects[projects['Project Grade Level Category']!='unknown']

In [38]:
assert donations.isna().sum().any() == 0

In [41]:
donors.isna().sum()#.any() == 0

Donor ID                 0
Donor City          213097
Donor State              0
Donor Is Teacher         0
Donor Zip           180060
dtype: int64

In [72]:
len(donors.dropna()) / len(donors)

0.899607564165379

In [73]:
donors.dropna(inplace=True)

In [59]:
donors.dropna(subset=['Donor Zip'], inplace=True)
donors.reset_index(inplace=True, drop=True)

In [60]:
donors.isna().sum()#.any() == 0

Donor ID                0
Donor City          33037
Donor State             0
Donor Is Teacher        0
Donor Zip               0
dtype: int64

In [74]:
assert donors.isna().sum().any() == 0

In [76]:
schools.isna().sum()

School ID                          0
School Name                        0
School Metro Type                  0
School Percentage Free Lunch    1141
School State                       0
School Zip                         0
School City                      227
School County                      2
School District                    0
dtype: int64

In [77]:
len(schools.dropna()) / len(schools)

0.9822037729645309

In [80]:
schools = schools.dropna().reset_index(drop=True)
assert schools.isna().sum().any() == 0

In [82]:
merge_1 = pd.merge(projects, donations, on='Project ID', how='inner')
merge_2 = pd.merge(merge_1, schools, on='School ID', how='inner')
merge_3 = pd.merge(merge_2, donors, on='Donor ID', how='inner')

In [83]:
del projects
del donors
del donations
del schools

In [84]:
del df_exp
del df_exp_1
del df_exp_3
del merge_1
del merge_2

In [85]:
gc.collect()

1256

In [ ]:
donations_projects = pd.merge(donations_0, projects_1, on='Project ID', how='inner')